# Processamento de Imagem e Visão
## Trabalho Prático 1
### Alunos: Belarmino Sacate (52057) e Miguel Ferreira (51878)

------
# 1. INTRODUÇÃO

-------------
## BIBLIOTECAS

In [291]:
import cv2
import numpy as np

------------
## 3.VARIAVEIS

In [292]:
ficheiro_video = 'Vídeo - Enunciado A.avi'

# parâmetros
area_minima_carro = 150
limiar_distancia = 60
historico_fundo = 40

# estados
veiculos_detetados_1 = {} 
veiculos_detetados_2 = {}  
veiculos_detetados_3 = {}   

outros_veiculos = {}

id_veiculos_contador = 0

contador_total = 0

# velocidade
fps = 25.0
pixels_por_metro = 5

In [293]:
# pts1 = [[25, 214],
#        [97, 85],
#        [158, 85],
#        [218, 231]]

# roi pontos
pts1 = [[24, 217], [78, 237], [114, 82], [99, 79]]
pts2 = [[83, 235], [145, 238], [134, 84], [118, 84]]
pts3 = [[152, 237], [138, 82], [155, 79], [218, 232]]

# array dos pontos
roi_selecionada_1 = np.array(pts1, dtype=np.int32)
roi_selecionada_2 = np.array(pts2, dtype=np.int32)
roi_selecionada_3 = np.array(pts3, dtype=np.int32)

# velocidades de cada faixa
faixa1 = []
faixa2 = []
faixa3 = []

------------
## METODOS

Estima a imagem de fundo

In [294]:
def obter_fundo(caminho_video, num_frames=50): 

    random_seed = np.random.seed(np.random.randint(0, 100))
    print(random_seed)

    # lista
    frames = []

    video = cv2.VideoCapture(caminho_video)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) # numero total de frames
    
    # escolhe frames aleatorios
    #random_frames = np.random.randint(0, total_frames, num_frames)

    # escolhe frames espaçadas igualmente
    intervalo = max(total_frames//num_frames, 1)

    for indice in range(0, total_frames, intervalo):
        # ler frames do video
        video.set(cv2.CAP_PROP_POS_FRAMES,indice)
        _, frame = video.read()
        frames.append(frame) # adiciona frame à lista
            
    video.release()
    
    # se nao houver frames
    if frames==[]:
        print("Falha ao ler frames para o modelo de fundo")

    # faz a mediana de todas as frames escolhidas (objetos que se movem são removidos)
    fundo = np.median(frames,axis=0).astype(dtype=np.uint8)

    print("Fundo gerado")
    return fundo

Subtração de fundo e binarização

In [295]:
def detetar_movimento(frame_cinza, fundo_cinza):
    # diferenca entre imagem e o fundo
    diferenca = cv2.absdiff(frame_cinza, fundo_cinza)

    # aplica threshold para binarizar
    _, binario = cv2.threshold(diferenca,30,255,cv2.THRESH_BINARY)
    return binario

Aplica operadores morfológicos (fecho/dilatação) e a máscara da ROI

In [296]:
def operadores_morfologicos(mascara_binaria, roi):
    
    kernel = np.ones((5,5), np.uint8)
    #kernel_ = np.ones((3,3), np.uint8)

    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    kernel3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    # kernel4 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    
    # operador MORPH_CLOSE
    mascara = cv2.morphologyEx(mascara_binaria, cv2.MORPH_CLOSE, kernel3)

    # dilatação
    mascara = cv2.dilate(mascara, kernel)
    
    # máscara da ROI
    mascara_roi = np.zeros_like(mascara)
    cv2.fillPoly(mascara_roi, [roi],255)

    # aplicar máscara da ROI
    mascara_processada = cv2.bitwise_and(mascara, mascara_roi)
    
    return mascara_processada

Detecção de regiões ativas (contornos)

In [ ]:
def detetar_veiculos(mascara_processada, area_minima):
    
    contornos, _ = cv2.findContours(mascara_processada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detecoes = []
    for contorno in contornos:

        #contorno
        area=cv2.contourArea(contorno)
 
        # comparar areas
        if area > area_minima:

            # calcular informacao do carro
            x, y, w, h = cv2.boundingRect(contorno)
            centro_x, centro_y = x + w // 2, y + h // 2
            detecoes.append((centro_x, centro_y, x, y, w, h))

    return detecoes

    Faz a correspondência das deteções com o histórico e atribui novos IDs.
    Retorna (deteções_com_id, novos_veiculos_rastreados, novo_contador_global).

In [ ]:
def atualizar_detecao(detecoes_atuais, historico_veiculos, limiar_distancia, contador_global_id, faixa,outros_veiculos):
    veiculos_novos = {}
    detecoes_id = []
    
    novo_contador_global_id = contador_global_id 
    
    # para todas as detecoes já registadas
    for centro_x, centro_y, x, y, w, h in detecoes_atuais:
        id_atual = -1
        velocidade = 0

        # se não encontrado noutra faixa, verificar se o veiculo ja esta registado na faixa atual
        for vid,dados in historico_veiculos.items():
            centro_x2, centro_y2 = dados['centro']
            
            # diferenca entre os centros
            distancia = np.sqrt((centro_x - centro_x2)**2 + (centro_y - centro_y2)**2)

            if distancia < limiar_distancia:
                id_atual = vid
                
                # velocidade
                distancia_m = distancia / pixels_por_metro
                tempo = 1.0 / fps
                velocidade = (distancia_m / tempo) * 3.6

                # velocidade por faixa
                if faixa == 1:
                    faixa1.append(velocidade)
                elif faixa == 2:
                    faixa2.append(velocidade)
                elif faixa == 3:
                    faixa3.append(velocidade)

                # registar no novo dicionario
                veiculos_novos[id_atual] = {
                    'centro': (centro_x, centro_y),
                    'velocidade': velocidade
                }
                break

        # ver se já existe noutra faixa
        # verificar se o veiculo ja esta registado noutra faixa
        if id_atual == -1:
            for vid, dados in outros_veiculos.items():
                centro_x2, centro_y2 = dados['centro']
                distancia = np.sqrt((centro_x - centro_x2)**2 + (centro_y - centro_y2)**2)

                if distancia < limiar_distancia:
                    id_atual = vid
                    velocidade = dados.get('velocidade', 0)
                    # registar no novo dicionario para manter tracking entre frames/faixas
                    veiculos_novos[id_atual] = {
                        'centro': (centro_x, centro_y),
                        'velocidade': velocidade
                    }
                    break
            
            # se não foi encontrado no historico, criar novo ID
            if id_atual == -1:
                novo_contador_global_id = novo_contador_global_id + 1
                id_atual = novo_contador_global_id

                # registar veiculo novo
                veiculos_novos[id_atual] = {
                    'centro': (centro_x, centro_y),
                    'velocidade': 0
                }
        
        detecoes_id.append((centro_x, centro_y, x, y, w, h, id_atual, velocidade))

    return detecoes_id, veiculos_novos, novo_contador_global_id

Desenha Bounding Boxes, IDs e contagem no frame

In [299]:
def desenhar_resultados(quadro_visual, deteccoes_com_id, roi_pontos, contagem_total):
    
    for centro_x, centro_y, x, y, w, h, id_atual, vel in deteccoes_com_id:
    
        # retangulo à volta do carro
        cv2.rectangle(quadro_visual,(x, y), (x + w, y + h),(0,0,255),2)
      
        # centro do carro
        cv2.circle(quadro_visual,(centro_x, centro_y), 5,(0,0,255),-1)

        # texto com id e velocidade
        cv2.putText(quadro_visual,f"V{id_atual} {int(vel)}km/h",(x,y-10), cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,0),2)

    # desenhar ROI
    cv2.polylines(quadro_visual,[roi_pontos],True, (0,255,0), 2)
    
    # total dos carros
    # cv2.putText(quadro_visual, f"total: {contagem_total}", (20,40), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)

EXECUÇÃO PRINCIPAL

In [ ]:
# inicializacao
imagem_fundo = obter_fundo(ficheiro_video, historico_fundo)
fundo_cinza = cv2.cvtColor(imagem_fundo, cv2.COLOR_BGR2GRAY)

captura = cv2.VideoCapture(ficheiro_video)

# reset
veiculos_detetados_1 = {}
veiculos_detetados_2 = {}
veiculos_detetados_3 = {}
id_veiculos_contador = 0
faixa1.clear()
faixa2.clear()
faixa3.clear()

# loop
while True:
    sucesso, frame = captura.read()

    if not sucesso:
        break 

    quadro_visual = frame.copy()
    quadro_cinza = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # guardar estados anteriores
    veiculos_anteriores_1 = veiculos_detetados_1.copy()
    veiculos_anteriores_2 = veiculos_detetados_2.copy()
    veiculos_anteriores_3 = veiculos_detetados_3.copy()

    # -------------------------------------------------------------------------------------------------------------------

    # AUTO-ESTRADA 1
    
    # Subtração do Fundo
    mascara_1 = detetar_movimento(quadro_cinza, fundo_cinza)
    
    # Operadores Morfologicos e ROI
    mascara_1 = operadores_morfologicos(mascara_1, roi_selecionada_1)
    
    # Deteção de Contornos
    detecoes_atuais_1 = detetar_veiculos(mascara_1, area_minima_carro)

    # Veículos de outras faixas
    outros_veiculos_1 = {**veiculos_detetados_2, **veiculos_detetados_3}
    
    # Atualização da Deteção
    resultados_deteccao_1, novos_veiculos_detetados_1, novo_id_veiculos_contador = atualizar_detecao(detecoes_atuais_1, veiculos_detetados_1, limiar_distancia, id_veiculos_contador, 1, outros_veiculos_1)

    # Atualização do Dicionário
    veiculos_detetados_1 = novos_veiculos_detetados_1
    id_veiculos_contador = novo_id_veiculos_contador

    # Visualização
    desenhar_resultados(quadro_visual, resultados_deteccao_1, roi_selecionada_1, id_veiculos_contador)

    # -------------------------------------------------------------------------------------------------------------------

    # AUTO-ESTRADA 2

    mascara_2 = detetar_movimento(quadro_cinza, fundo_cinza)
    mascara_2 = operadores_morfologicos(mascara_2, roi_selecionada_2)
    detecoes_atuais_2 = detetar_veiculos(mascara_2, area_minima_carro)

    # Usar veículos anteriores + zona 1 já atualizada
    outros_veiculos_2 = {**veiculos_detetados_1, **veiculos_anteriores_3}

    resultados_deteccao_2, novos_veiculos_detetados_2, novo_id_veiculos_contador = atualizar_detecao(detecoes_atuais_2, veiculos_anteriores_2, limiar_distancia, id_veiculos_contador, 2, outros_veiculos_2)

    veiculos_detetados_2 = novos_veiculos_detetados_2
    id_veiculos_contador = novo_id_veiculos_contador

    desenhar_resultados(quadro_visual, resultados_deteccao_2, roi_selecionada_2, id_veiculos_contador)
    
    # -------------------------------------------------------------------------------------------------------------------

    # AUTO-ESTRADA 3

    mascara_3 = detetar_movimento(quadro_cinza, fundo_cinza)
    mascara_3 = operadores_morfologicos(mascara_3, roi_selecionada_3)
    detecoes_atuais_3 = detetar_veiculos(mascara_3, area_minima_carro)

    outros_veiculos_3 = {**veiculos_detetados_1, **veiculos_detetados_2}

    resultados_deteccao_3, novos_veiculos_detetados_3, novo_id_veiculos_contador = atualizar_detecao(detecoes_atuais_3, veiculos_anteriores_3, limiar_distancia, id_veiculos_contador, 3, outros_veiculos_3)

    veiculos_detetados_3 = novos_veiculos_detetados_3
    id_veiculos_contador = novo_id_veiculos_contador
    
    desenhar_resultados(quadro_visual, resultados_deteccao_3, roi_selecionada_3, id_veiculos_contador)

    # -------------------------------------------------------------------------------------------------------------------

    # total dos carros
    cv2.putText(quadro_visual, f"total: {id_veiculos_contador}", (20,40), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,0), 2)

    # media de velocidade
    if faixa1:
        v_media_1 = np.mean(faixa1)
    else:
        v_media_1 = 0

    if faixa2:
        v_media_2 = np.mean(faixa2)
    else:
        v_media_2 = 0
    
    if faixa3:
        v_media_3 = np.mean(faixa3)
    else:
        v_media_3 = 0
    
    # display
    cv2.putText(quadro_visual, f"v media 1: {np.round(v_media_1,2)}km/h", (140,20), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255), 1)
    cv2.putText(quadro_visual, f"v media 2: {np.round(v_media_2,2)}km/h", (140,40), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255), 1)
    cv2.putText(quadro_visual, f"v media 3: {np.round(v_media_3,2)}km/h", (140,60), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255), 1)

    cv2.imshow("Contagem de Veiculos", quadro_visual)

    mascara_total= np.zeros_like(quadro_cinza)
    mascara_total= (mascara_1 | mascara_2 | mascara_3)

    cv2.imshow("Mascara Total", mascara_total)

    # slow down
    if cv2.waitKey(20)==27:
        break

# finalizacao
captura.release()
cv2.destroyAllWindows()
print("Processamento finalizado.")

print("total de veiculos detectados: ", id_veiculos_contador)

None
Fundo gerado
Processamento finalizado.
total de veiculos detectados:  13


------------
# 5. CONCLUSÃO